## PE & NN compression

## Compression Techniques

for fc/conv layers

for activations

## Methodology
 
Evaluation of:
- accuracy
- complexity

## Experiment1

Static architecture shape

## Experiment2

NAS with achitecture & compression techniques

## Source code

1. Available

## Technical notes

1. MNIST didn't run as expected...
2. Bias is disabled in conv layers.

## Next Steps:

- Converting to a library
- Per-conv-layer activation type OR per-conv-layer add_activation boolean
- Integrating with other compression NN tools
- BiPer
- RBNN
- TBN
- 10+ other papers worth familiarizing with

In [1]:
from src.datasets.mnist_dataset import MiniMNISTDataset
from src.plot_experiment2 import cnn_report_to_df


df = cnn_report_to_df("../../reports/mini-mnist-v3", MiniMNISTDataset)
df

,acc,cost,in_bitwidth,conv_layers,conv_channels1,conv_stride1,conv_pooling_size1,conv_compression_bitwidth1,conv_channels2,conv_stride2,...,conv_compression,fc_compression,dropout,activation,activation_qmode,activation_reste_o,activation_reste_threshold,learning_rate,weight_decay,chromosome
0,95.700,21383736.0,3,4,16,1,1,8,128,1,...,NBITS,NONE,0.2,RELU,QMode.DET,3,1.5,0.001,0.00001,"CNNChromosome(in_bitwidth=3, conv_layers=4, co..."
1,95.425,1031821.0,6,1,128,1,2,5,32,1,...,BINARY,NONE,0.2,RELU,QMode.DET,3,1.5,0.001,0.00001,"CNNChromosome(in_bitwidth=6, conv_layers=1, co..."
2,92.625,11684.0,7,1,32,2,2,3,64,1,...,TERNARY,BINARY,0.2,NONE,QMode.DET,3,1.5,0.001,0.00001,"CNNChromosome(in_bitwidth=7, conv_layers=1, co..."
3,93.900,385028.0,8,3,16,1,1,6,32,2,...,TERNARY,NBITS,0.0,NONE,QMode.DET,3,1.5,0.001,0.00001,"CNNChromosome(in_bitwidth=8, conv_layers=3, co..."


In [2]:
chromosomes = []

for idx, row in df.iterrows():
    chromosomes.append(row["chromosome"])

In [ ]:

from src.nas.chromosome import ChromosomeConfig
from src.nas.cnn_chromosome import CNNChromosome
from src.nas.cnn_nas_problem import CnnNasProblem
from src.nas.nas_params import NasParams


cfg = ChromosomeConfig(CNNChromosome)
nas_params = NasParams(
    batch_size=50,
    epochs=30,
    patience=5,
    amount_of_evaluations=3,
    population_size=40,
    population_offspring_count=8,
    algorithm_generations=1,
)
nas_problem = CnnNasProblem(nas_params, MiniMNISTDataset)

ch = chromosomes[3]

params = nas_problem.get_nn_params(ch)
model = params.get_model()
model

CNN(
  (quantize_input): Quantize()
  (conv_layers): ModuleList(
    (0): Sequential(
      (0): Conv2dTernary(1, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Identity()
    )
    (1): Sequential(
      (0): Conv2dTernary(16, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Identity()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2dTernary(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Identity()
    )
  )
  (fc_layers): Sequential(
    (0): LinearQunatized(in_features=256, out_features=64, nbits=2, bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [ ]:
# Load a stored model & evaluate its accuracy

from src.models.eval import NNArchitectureAccuracyEvaluator
from src.models.nn import load_model

filename = "93.9_7-2-0-0-0-5-2-1-1-1-3-1-0-7-4-1-0-0-2-3-1-3-5-2-1-6-1-2-0-0-0-0-0-0-0.pt"

model = load_model(model, "../../reports/mini-mnist-v3/models/" + filename)

acc_eval = NNArchitectureAccuracyEvaluator(params)
acc_eval.test_model(model)

97.875

In [ ]:
# Train a model with a given architecture

from src.models.eval import NNArchitectureEvaluator


evaluator = NNArchitectureEvaluator(params.train)
evaluator.evaluate_complexity(params)

385028.0

In [13]:
evaluator.evaluate_accuracy(params)

{'max': np.float64(94.05),
 'mean': np.float64(94.05),
 'std': np.float64(0.0),
 'accuracies': array([94.05]),
 'best_model': CNN(
   (quantize_input): Quantize()
   (conv_layers): ModuleList(
     (0): Sequential(
       (0): Conv2dTernary(1, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
       (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): Identity()
     )
     (1): Sequential(
       (0): Conv2dTernary(16, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): Identity()
       (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     )
     (2): Sequential(
       (0): Conv2dTernary(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): Identity()
     )
   )
   (fc_layers): Sequential(
     (0): Lin